In [1]:
# Importação de bibliotecas

#Avisos
import warnings
warnings.filterwarnings('ignore')

# DOE
from pyDOE2 import ccdesign, fullfact
import itertools

# Análises
import numpy as np
import pandas as pd

# Plots
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D
import plotly.express as px

#Análises Estatísticas
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from factor_analyzer import FactorAnalyzer
from scipy.stats import qmc
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.regression.linear_model import OLS
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.anova import anova_lm
from scipy import stats
from scipy.stats import t

#Otimização
from scipy.optimize import minimize
from itertools import combinations
from itertools import combinations_with_replacement
from deap import algorithms, base, creator, tools
import random
import numpy as np
from scipy.spatial.distance import mahalanobis
from pymoo.core.problem import ElementwiseProblem
#from pymoo.optimize import minimize
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.operators.crossover.sbx import SimulatedBinaryCrossover
from pymoo.operators.mutation.pm import PolynomialMutation
from pymoo.termination import get_termination
import plotly.graph_objs as go
from matplotlib.patches import Ellipse
import itertools
import matplotlib.transforms as transforms
import seaborn as sns
from scipy.cluster import hierarchy
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial import distance
from sklearn.preprocessing import MinMaxScaler

In [33]:
# NSGA-II

from pymoo.optimize import minimize

# Dados do problema
data = np.array([
    [66, 68.306, 4.492, 0.62, 3, 3.66, 7.587, 52.991],
    [33, 33.785, 8.886, 0.64, 3, 2.33, 16.939, 39.172],
    [54, 54.742, 5.835, 2.14, 7.26, 3.47, 18.219, 39.418],
    [30, 32.362, 11.404, 2.15, 7.42, 2.17, 28.881, 24.936],
    [63, 65.843, 4.453, 0.65, 3.02, 5.2, 7.642, 52.963],
    [31.5, 32.362, 9.629, 0.62, 2.84, 3.01, 16.5, 39.142],
    [53, 53.808, 6.626, 1.93, 6.78, 4.61, 18.25, 39.411],
    [31.5, 33.578, 10.964, 2.39, 8.28, 1.79, 28.881, 24.95],
    [60.5, 62.672, 4.252, 0.67, 2.9, 5.51, 6.745, 53.927],
    [25.5, 27.458, 12.715, 1.17, 4.62, 1.87, 26.205, 27.406],
    [40, 42.432, 7.046, 0.31, 1.84, 4.32, 5.602, 53.596],
    [41.5, 43.498, 8.358, 2.14, 7.44, 2.58, 27.512, 27.561],
    [51.5, 52.202, 6.423, 1.2, 4.56, 2.28, 20.659, 36.508],
    [48.5, 49.169, 5.761, 0.7, 3.1, 3.46, 20.5, 36.486],
    [43.5, 44.284, 7.117, 0.81, 3.94, 3.47, 20.31, 36.444],
    [44.5, 45.284, 6.673, 0.82, 3.15, 3.42, 20.284, 36.453],
    [42.5, 43.284, 6.516, 0.83, 3.32, 3.53, 20.336, 36.434],
    [43.25, 44.034, 6.505, 0.81, 3.96, 3.41, 20.257, 36.441],
    [44, 44.784, 6.47, 0.83, 3.82, 3.36, 20.097, 36.72]
])

# Valores de Utopia e Nadir
utopia = np.array([65.421, 68.067, 0.012, 2.263, 7.895, 5.572, 0.304, 0.570])
nadir = np.array([28.431, 30.800, 0.004, 0.281, 1.847, 1.711, 0.043, 0.231])

# Definir a classe do problema
class MyProblem(ElementwiseProblem):
    def __init__(self, utopia, nadir):
        super().__init__(n_var=8, n_obj=8, n_constr=0, xl=nadir, xu=utopia)
        self.utopia = utopia
        self.nadir = nadir

    def _evaluate(self, x, out, *args, **kwargs):
        T, MTTF, WR, Ra, Rt, Kp, ROI, OEE = x
        out["F"] = [-T, -MTTF, WR, Ra, Rt, Kp, -ROI, -OEE]

# Criar uma instância do problema
problem = MyProblem(utopia, nadir)

# Configuração do algoritmo NSGA-II
algorithm = NSGA2(
    pop_size=50,
    sampling=FloatRandomSampling(),
    crossover=SimulatedBinaryCrossover(prob=0.9, eta=15),
    mutation=PolynomialMutation(eta=20),
    eliminate_duplicates=True
)

# Critério de terminação
termination = get_termination("n_gen", 50)

# Executar a otimização
res = minimize(problem,
               algorithm,
               termination,
               seed=1,
               save_history=True,
               verbose=True)

# Exibir os resultados
print("Melhores indivíduos após a evolução:")

# Preparar os dados para o DataFrame
df_results = pd.DataFrame(res.X, columns=['T', 'MTTF', 'WR', 'Ra', 'Rt', 'Kp', 'ROI', 'OEE'])

# Exibir o DataFrame
print(round(df_results.head(), 3))

# Salvar os resultados em um arquivo Excel
df_results.to_excel("resultadosotimizacaonsga.xlsx", index=False)
print("Resultados salvos em 'resultadosotimizacaonsga.xlsx'")

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |       50 |     49 |             - |             -
     2 |      100 |     50 |  0.0049330415 |         ideal
     3 |      150 |     50 |  0.0096907934 |         ideal
     4 |      200 |     50 |  0.0025861358 |         ideal
     5 |      250 |     50 |  0.0089878696 |         ideal
     6 |      300 |     50 |  0.0195010664 |         nadir
     7 |      350 |     50 |  0.0081316980 |         ideal
     8 |      400 |     50 |  0.0237526111 |         nadir
     9 |      450 |     50 |  0.0310018584 |         ideal
    10 |      500 |     50 |  0.0040519511 |         ideal
    11 |      550 |     50 |  0.0107951575 |         ideal
    12 |      600 |     50 |  0.0028295403 |         ideal
    13 |      650 |     50 |  0.1605727619 |             f
    14 |      700 |     50 |  0.1664129832 |             f
    15 |      750 |     50 |  0.0033708167 |         nadir
    16 |      800 |     50 |  0.1539837957 |            

In [41]:
# NSGA-II (Fatores)

from pymoo.optimize import minimize

# Dados do problema para otimização dos fatores VRF1, VRF2, VRF3
data = np.array([
    [-1.728, 0.677, -0.039],
    [1.068, 1.013, 0.190],
    [-0.831, -1.646, -0.452],
    [1.103, -1.398, 0.354],
    [-1.214, 0.274, -1.303],
    [1.526, 0.849, -0.563],
    [-0.352, -1.675, -1.335],
    [0.947, -1.751, 0.444],
    [-0.887, 0.216, -1.785],
    [1.849, 0.157, 0.517],
    [1.057, 1.032, -2.114],
    [-0.204, -1.400, 0.830],
    [-1.093, 0.245, 1.581],
    [-0.671, 0.793, 1.038],
    [-0.064, 0.429, 0.433],
    [-0.158, 0.646, 0.622],
    [0.019, 0.573, 0.428],
    [-0.054, 0.440, 0.459],
    [-0.313, 0.527, 0.692]
])

# Valores de Utopia e Nadir
utopia = np.array([1.658, 1.217, 0.760])
nadir = np.array([-1.276, -1.556, -1.752])

# Definir a classe do problema
class MyProblem(ElementwiseProblem):
    def __init__(self, utopia, nadir):
        super().__init__(n_var=3, n_obj=3, n_constr=0, xl=nadir, xu=utopia)
        self.utopia = utopia
        self.nadir = nadir

    def _evaluate(self, x, out, *args, **kwargs):
        VRF1, VRF2, VRF3 = x
        out["F"] = [-VRF1, -VRF2, -VRF3]

# Criar uma instância do problema
problem = MyProblem(utopia, nadir)

# Configuração do algoritmo NSGA-II
algorithm = NSGA2(
    pop_size=50,
    sampling=FloatRandomSampling(),
    crossover=SimulatedBinaryCrossover(prob=0.9, eta=15),
    mutation=PolynomialMutation(eta=20),
    eliminate_duplicates=True
)

# Critério de terminação
termination = get_termination("n_gen", 50)

# Executar a otimização
res = minimize(problem,
               algorithm,
               termination,
               seed=1,
               save_history=True,
               verbose=True)

# Exibir os resultados
print("Melhores indivíduos após a evolução:")

# Preparar os dados para o DataFrame
df_results = pd.DataFrame(res.X, columns=['VRF1', 'VRF2', 'VRF3'])

# Exibir o DataFrame
print(round(df_results.head(), 3))

# Salvar os resultados em um arquivo Excel
df_results.to_excel("resultadosotimizacaonsgavrf.xlsx", index=False)
print("Resultados salvos em 'resultadosotimizacaonsgavrf.xlsx'")

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |       50 |     12 |             - |             -
     2 |      100 |     18 |  0.0311360678 |         ideal
     3 |      150 |     17 |  2.0437269078 |         nadir
     4 |      200 |     25 |  0.0090630309 |         ideal
     5 |      250 |     23 |  0.0140532672 |         ideal
     6 |      300 |     19 |  0.0168658098 |         ideal
     7 |      350 |     13 |  0.4623548440 |         nadir
     8 |      400 |     15 |  0.0441855977 |         nadir
     9 |      450 |     19 |  1.4213636413 |         nadir
    10 |      500 |     21 |  0.6943927762 |         nadir
    11 |      550 |     21 |  1.9947786174 |         nadir
    12 |      600 |     30 |  0.6686547917 |         nadir
    13 |      650 |     32 |  1.2622309309 |         nadir
    14 |      700 |     32 |  0.5807253776 |         nadir
    15 |      750 |     30 |  0.1905991303 |         nadir
    16 |      800 |     34 |  0.1626393187 |         nad

In [3]:
# TOPSIS (Variáveis)

import pandas as pd
import numpy as np

def topsis(data, weights, impacts):
    # Normalize the decision matrix
    norm_data = data / np.sqrt((data ** 2).sum(axis=0))
    
    # Multiply the normalized decision matrix by the weights
    weighted_data = norm_data * weights
    
    # Identify the ideal best and worst
    ideal_best = np.max(weighted_data * impacts, axis=0)
    ideal_worst = np.min(weighted_data * impacts, axis=0)
    
    # Calculate the Euclidean distance from the ideal best and worst
    distance_best = np.sqrt(((weighted_data - ideal_best) ** 2).sum(axis=1))
    distance_worst = np.sqrt(((weighted_data - ideal_worst) ** 2).sum(axis=1))
    
    # Calculate the TOPSIS score
    topsis_score = distance_worst / (distance_best + distance_worst)
    
    return topsis_score

# Load the data from the Excel file
file_path = 'topsisv.xlsx'
data = pd.read_excel(file_path)

# Define the weights and impacts manually
weights = np.array([1, 1, 1, 1, 1, 1, 1, 1])  # Adjust the weights as needed
impacts = np.array([1, 1, -1, -1, -1, -1, 1, 1])  # 1 for positive impact, -1 for negative impact

decision_matrix = data.iloc[:, :]

# Calculate TOPSIS score
topsis_scores = topsis(decision_matrix.values, weights, impacts)

# Add the TOPSIS scores to the DataFrame
data['TOPSIS'] = topsis_scores

# Save the results back to the Excel file
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
    data.to_excel(writer, index=False, sheet_name='TOPSIS')

print("TOPSIS calculation completed and saved to 'topsisv.xlsx'.")

TOPSIS calculation completed and saved to 'topsisv.xlsx'.


In [42]:
# TOPSIS (Fatores)

import pandas as pd
import numpy as np

def topsis(data, weights, impacts):
    # Normalize the decision matrix
    norm_data = data / np.sqrt((data ** 2).sum(axis=0))
    
    # Multiply the normalized decision matrix by the weights
    weighted_data = norm_data * weights
    
    # Identify the ideal best and worst
    ideal_best = np.max(weighted_data * impacts, axis=0)
    ideal_worst = np.min(weighted_data * impacts, axis=0)
    
    # Calculate the Euclidean distance from the ideal best and worst
    distance_best = np.sqrt(((weighted_data - ideal_best) ** 2).sum(axis=1))
    distance_worst = np.sqrt(((weighted_data - ideal_worst) ** 2).sum(axis=1))
    
    # Calculate the TOPSIS score
    topsis_score = distance_worst / (distance_best + distance_worst)
    
    return topsis_score

# Load the data from the Excel file
file_path = 'topsisf.xlsx'
data = pd.read_excel(file_path)

# Define the weights and impacts manually
weights = np.array([1, 1, 1])  # Adjust the weights as needed
impacts = np.array([1, 1, 1])  # 1 for positive impact, -1 for negative impact

decision_matrix = data.iloc[:, :]

# Calculate TOPSIS score
topsis_scores = topsis(decision_matrix.values, weights, impacts)

# Add the TOPSIS scores to the DataFrame
data['TOPSIS'] = topsis_scores

# Save the results back to the Excel file
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
    data.to_excel(writer, index=False, sheet_name='TOPSIS')

print("TOPSIS calculation completed and saved to 'topsisf.xlsx'.")

TOPSIS calculation completed and saved to 'topsisf.xlsx'.


In [4]:
import pandas as pd
import numpy as np

# Carregar os dados da planilha
file_path = 'topsisv.xlsx'
df = pd.read_excel(file_path)

# Verificar as colunas da planilha
columns = df.columns

# Definir os critérios e suas direções de otimização
criteria = ['T', 'MTTF', 'Wr', 'Ra', 'Rt', 'Kp', 'ROI', 'OEE']
directions = ['max', 'max', 'min', 'min', 'min', 'min', 'max', 'max']

# Normalizar a matriz de decisão
normalized_matrix = df[criteria].copy()
for i, criterion in enumerate(criteria):
    if directions[i] == 'max':
        normalized_matrix[criterion] = df[criterion] / np.sqrt((df[criterion] ** 2).sum())
    else:
        normalized_matrix[criterion] = (1 / df[criterion]) / np.sqrt((1 / df[criterion] ** 2).sum())

# Ponderar a matriz normalizada com pesos iguais
weights = np.ones(len(criteria)) / len(criteria)
weighted_matrix = normalized_matrix * weights

# Determinar as soluções ideais positiva e negativa
ideal_positive = weighted_matrix.max()
ideal_negative = weighted_matrix.min()

# Calcular a distância de cada alternativa para as soluções ideais positiva e negativa
dist_positive = np.sqrt(((weighted_matrix - ideal_positive) ** 2).sum(axis=1))
dist_negative = np.sqrt(((weighted_matrix - ideal_negative) ** 2).sum(axis=1))

# Calcular a pontuação TOPSIS para cada alternativa
topsis_score = dist_negative / (dist_positive + dist_negative)

# Adicionar os resultados na planilha
df['TOPSIS Score'] = topsis_score

# Salvar os resultados no mesmo arquivo de Excel
df.to_excel(file_path, index=False)

print('TOPSIS calculation completed and results saved to', file_path)

TOPSIS calculation completed and results saved to topsisv.xlsx
